### torchtext 라이브러리로 텍스트 분류
- (1)단계 - 데이터 전처리 : 숫자형식으로 변환하는 것 까지
- (2)단계 - 모델 구현

(1-1) 데이터 준비 => 내장 데이터셋 활용
- AG_NEWS 데이터셋 반복자 : 레이블(label) + 문장의 튜플(tuple) 형태

In [1]:
# !pip install torchdata

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/172.0 kB ? eta -:--:--
   --------- ----------------------------- 41.0/172.0 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 172.0/172.0 kB 2.6 MB/s eta 0:00:00


In [2]:
import torchdata

In [3]:
import torch
from torchtext.datasets import AG_NEWS

# DataPipe 타입 >>> iterator 타입 형변환
train_iter = iter(AG_NEWS(split='train'))

In [15]:
cnt = 0
for a in AG_NEWS(split='train'):
    cnt += 1
    print(a)
    if cnt == 10: break

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")
(3, 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.')
(3, "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.")
(3, 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.')
(3, 'Oil prices soar to all-time record, posing new menace t

In [4]:
# 데이터 확인 => (label, text), label 1~4
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

### (2) 데이터 처리 파이프라인 준비
- 어휘집(vocab), 단어 벡터(word vector), 토크나이저(tokenizer)
- 가공되지 않은 텍스트 문자열에 대한 데이터 처리 빌딩 블록
- 일반적인 NLP 데이터 처리
    - 첫번째 단계 : 가공되지 않은 학습 데이터셋으로 어휘집 생성
        => 토큰 목록 또는 반ㅂ족자 받는 내장 팩토리 함수(factory function) : build_vocab_from_iterator
    - 사용자는 어휘집에 추가할 특수 기호(special symbol) 전달 가능

In [5]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

### 토크나이저 생성
tokenizer = get_tokenizer('basic_english')

### 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split='train')

In [6]:
### 토큰 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트 -> 텍스트 토큰화
        yield tokenizer(text)

In [7]:
### 단어사전 생성 (조각난 단어를 숫자로 바꾸는 역할)
vocab = build_vocab_from_iterator(yield_tokens(train_iter),
                                  specials=['<unk>'])

### <UNK> 인덱스 0으로 설정
vocab.set_default_index(vocab['<unk>'])

In [8]:
### 숫자 배정 방식 => 빈도 (3이다? 그 말은 3번째로 많이 등장한 단어)
vocab(['<unk>', 'here', 'is', 'an', 'example'])

[0, 475, 21, 30, 5297]

In [10]:
### 텍스트 >>> 정수 인코딩
text_pipeline = lambda x: vocab(tokenizer(x))

### 레이블 >>> 정수 인코딩
label_pipeline = lambda x: int(x) - 1

- (3) 데이터 배치(batch)와 반복자 생성
    - torch.utils.data.DataLoader : getitem(), len() 구현한 맵 형태(map-style)
    - collate_fn() : DataLoader로부터 생성된 샘플 배치 함수
        - 입력 : DataLoader에 배치 크기(batch size)가 있는 배치(batch) 데이터

In [11]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### DataLoader에서 배치크기만큼 데이터셋 반환하는 함수
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]    # offset : 파일 포인터
    
    # 뉴스기사, 라벨을 1개씩 추출해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))
    
    # 텐서화 진행
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

In [16]:
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter,
                        batch_size=8,
                        shuffle=True,
                        collate_fn=collate_batch)

In [12]:
### 분류 클래스 수와 단어사전 개수
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f'num_class : {num_class}     vocab_size : {vocab_size}')

num_class : 4     vocab_size : 95811


In [18]:
for labels, texts, offsets in dataloader:
    print(labels, texts, offsets)
    break

tensor([2, 2, 1, 3, 3, 1, 3, 3]) tensor([ 3870,     4,   275,  5694,  1653,  3870,   691,     6,  1524,    26,
           55,    25,    33,  4185,  5694,  1486,  1653,     4,   378,    43,
         2321,    11,    46,  5480,  6492,     1,     2,  3400,  1363,    26,
            2,  1969,   648,   393,    34,   299,     6, 18098,     3,   202,
         1189, 10312,     2, 17448,     7,     2,   294,    12,     9,    47,
          132,     6,   633,     3,   360,  2073,     3,   982,     8, 14922,
          187,    20,     5,  6597,  1267,     3,    18,     2,   342,  1395,
          177,   945,  3357,    15,   640,   106,   945,   390,    25,  1292,
            1,   193,   109,   554,  2946,   193,    12,     9, 42413,   111,
           39, 12935, 12004,   566, 18550,     4,   109,     2,   313,   554,
         2946,   707,     3,  8019,   123,     6,    40,  5151,   458,     5,
         2344,  2014,  5158,   296,     4,   270,     1,   657,   210,   444,
          606,    38,   193,   

c:\ProgramData\anaconda3\envs\Text_017_220_38\lib\site-packages\torch\utils\data\datapipes\iter\combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "
